## 🧪 Spark Streaming 로직 테스트 노트북
실시간 스트리밍을 돌리기 전에, Kafka에서 데이터를 잘 가져오는지, 파싱은 잘 되는지 확인하는 용도입니다.

In [ ]:
# 1. 라이브러리 임포트 및 세션 생성
import os
import sys
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

# 필수 패키지 로딩 (Kafka, PostgreSQL)
spark = SparkSession.builder \
    .appName("TestNotebook") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0,org.postgresql:postgresql:42.6.0") \
    .getOrCreate()

print("Spark Session Created!")

In [ ]:
# 2. 스키마 정의 (streaming_analysis.py와 동일해야 함)
order_details_schema = StructType() \
    .add("order_id", StringType()) \
    .add("user_id", StringType()) \
    .add("product_id", StringType()) \
    .add("category", StringType()) \
    .add("quantity", IntegerType()) \
    .add("total_amount", IntegerType()) \
    .add("payment_method", StringType()) \
    .add("user_region", StringType()) \
    .add("user_age_group", StringType()) \
    .add("created_at", TimestampType())

final_schema = StructType() \
    .add("event_type", StringType()) \
    .add("timestamp", StringType()) \
    .add("order", order_details_schema)

print("Schema Defined!")

In [ ]:
# 3. Kafka 데이터 읽기 (Batch Mode)
# 중요: readStream이 아니라 read를 사용합니다. (현재 쌓여있는 데이터만 가져옴)
df_raw = spark.read \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka1:29092,kafka2:29093,kafka3:29094") \
    .option("subscribe", "orders") \
    .option("startingOffsets", "earliest") \
    .load()

print(f"Raw Data Count: {df_raw.count()}")

In [ ]:
# 4. 데이터 파싱 테스트
# 여기서 null이 나오면 스키마가 틀린 것!
df_parsed = df_raw.selectExpr("CAST(value AS STRING) as json_str") \
    .select(from_json(col("json_str"), final_schema).alias("data")) \
    .select("data.order.*")

# 데이터 눈으로 확인
df_parsed.show(5, truncate=False)
df_parsed.printSchema()

In [ ]:
# 5. [테스트 1] 카테고리별 매출 집계 테스트
# 윈도우 집계가 잘 되는지 확인
df_category = df_parsed \
    .groupBy(window("created_at", "1 minute"), "category") \
    .agg(count("order_id").alias("count"), sum("total_amount").alias("revenue")) \
    .orderBy("window", "category")

print("=== 카테고리별 집계 결과 ===")
df_category.show(5, truncate=False)

In [ ]:
# 6. [테스트 2] 연령대별 집계 테스트
df_age = df_parsed \
    .groupBy("user_age_group") \
    .agg(count("order_id").alias("count")) \
    .orderBy("user_age_group")

print("=== 연령대별 주문 수 ===")
df_age.show()

### ✅ 테스트 완료
위 결과가 정상적으로 나온다면 `streaming_analysis.py`를 실행해도 좋습니다.